In [1]:
!pip install kaggle

In [0]:
!export KAGGLE_USERNAME=parthpartani
!export KAGGLE_KEY=f131feed2dd3ba88ad0100c3e5cdadbd

In [5]:
from google.colab import files

#retrieve uploaded file
uploaded = files.upload()

#print results
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 68 bytes


In [6]:
!kaggle competitions download -c career-con-2019

 76% 13.0M/17.1M [00:00<00:00, 63.8MB/s]
100% 17.1M/17.1M [00:00<00:00, 67.3MB/s]
  0% 0.00/64.1k [00:00<?, ?B/s]
100% 64.1k/64.1k [00:00<00:00, 19.8MB/s]
  0% 0.00/51.1k [00:00<?, ?B/s]
100% 51.1k/51.1k [00:00<00:00, 44.2MB/s]
 52% 9.00M/17.2M [00:00<00:00, 41.9MB/s]
100% 17.2M/17.2M [00:00<00:00, 57.0MB/s]


In [0]:
import numpy as np
import pandas as pd

In [0]:
x_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")
X_test = pd.read_csv("X_test.csv")

In [0]:
def feature_extraction(raw_frame):
    frame = pd.DataFrame()
    raw_frame['orientation'] = raw_frame['orientation_X'] + raw_frame['orientation_Y'] + raw_frame['orientation_Z']+ raw_frame['orientation_W']
    
    raw_frame['angular_velocity'] = raw_frame['angular_velocity_X'] + raw_frame['angular_velocity_Y'] + raw_frame['angular_velocity_Z']
    raw_frame['linear_acceleration'] = raw_frame['linear_acceleration_X'] + raw_frame['linear_acceleration_Y'] + raw_frame['linear_acceleration_Y']
    raw_frame['velocity_to_acceleration'] = raw_frame['angular_velocity'] / raw_frame['linear_acceleration']
    raw_frame['velocity_linear_acceleration'] = raw_frame['linear_acceleration'] * raw_frame['angular_velocity']
    # print(raw_frame.columns[3])
    for col in raw_frame.columns[3:]:
        frame[col + '_mean'] = raw_frame.groupby(['series_id'])[col].mean()
        frame[col + '_std'] = raw_frame.groupby(['series_id'])[col].std()
        frame[col + '_max'] = raw_frame.groupby(['series_id'])[col].max()
        frame[col + '_min'] = raw_frame.groupby(['series_id'])[col].min()
        frame[col + '_max_to_min'] = frame[col + '_max'] / frame[col + '_min']
        frame[col + '_mean_abs_change'] = raw_frame.groupby('series_id')[col].apply(lambda x: np.mean(np.abs(np.diff(x))))
        frame[col + '_abs_max'] = raw_frame.groupby('series_id')[col].apply(lambda x: np.max(np.abs(x)))
    return frame

In [0]:
train = feature_extraction(x_train)

In [86]:
test = feature_extraction(X_test)
train.head()

,orientation_X_mean,orientation_X_std,orientation_X_max,orientation_X_min,orientation_X_max_to_min,orientation_X_mean_abs_change,orientation_X_abs_max,orientation_Y_mean,orientation_Y_std,orientation_Y_max,...,velocity_to_acceleration_max_to_min,velocity_to_acceleration_mean_abs_change,velocity_to_acceleration_abs_max,velocity_linear_acceleration_mean,velocity_linear_acceleration_std,velocity_linear_acceleration_max,velocity_linear_acceleration_min,velocity_linear_acceleration_max_to_min,velocity_linear_acceleration_mean_abs_change,velocity_linear_acceleration_abs_max
series_id,,,,,,,,,,,,,,,,,,,,,
0,-0.758666,0.000363,-0.75822,-0.75953,0.998275,0.000015,0.75953,-0.634008,0.000471,-0.63306,...,-14.980095,0.057500,2.444038,0.014491,0.330151,1.346336,-1.025277,-1.313143,0.231566,1.346336
1,-0.958606,0.000151,-0.95837,-0.95896,0.999385,0.000023,0.95896,0.241867,0.000499,0.24270,...,-4.478137,0.057504,1.157278,0.057979,0.719351,2.305173,-2.717101,-0.848394,0.517250,2.717101
2,-0.512057,0.001377,-0.50944,-0.51434,0.990473,0.000041,0.51434,-0.846171,0.000785,-0.84490,...,-13.212587,0.111821,4.445092,0.137865,0.506150,1.846611,-0.980432,-1.883467,0.311430,1.846611
3,-0.939169,0.000227,-0.93884,-0.93968,0.999106,0.000026,0.93968,0.310140,0.000453,0.31147,...,-0.373159,0.086081,1.437307,0.088078,1.825780,5.313508,-7.071409,-0.751407,1.185604,7.071409
4,-0.891301,0.002955,-0.88673,-0.89689,0.988672,0.000080,0.89689,0.428144,0.006165,0.43740,...,-0.069095,0.007255,0.166277,-0.523060,0.398815,0.286279,-1.553462,-0.184285,0.149926,1.553462


In [157]:
  train.shape

(3810, 105)

In [0]:
label=pd.read_csv("y_train.csv")
label=label["surface"]

In [0]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(label)
label=label_encoder.transform(label)

In [89]:
label_encoder.inverse_transform(label)

array(['fine_concrete', 'concrete', 'concrete', ..., 'fine_concrete',
       'tiled', 'soft_pvc'], dtype=object)

In [0]:
Y_train = label
X_train = train
test=test
X_train.shape, Y_train.shape
scaler = StandardScaler()
scaler.fit(X_train)
# Apply transform to both the training set and the test set.
train_img = scaler.transform(X_train)
test_img = scaler.transform(test)

In [132]:
type(Y_train)

numpy.ndarray

In [0]:
para={'boosting_type': 'gbdt',
 'colsample_bytree': 0.85,
 'learning_rate': 0.1,
 'max_bin': 512,
 'max_depth': -1,
 'metric': 'multi_error',
 'min_child_samples': 8,
 'min_child_weight': 1,
 'min_split_gain': 0.5,
 'nthread': 3,
 'num_class': 9,
 'num_leaves': 31,
 'objective': 'multiclass',
 'reg_alpha': 0.8,
 'reg_lambda': 1.2,
 'scale_pos_weight': 1,
 'subsample': 0.7,
 'subsample_for_bin': 200,
 'subsample_freq': 1}

In [145]:
from sklearn.model_selection import GridSearchCV,StratifiedKFold, train_test_split
import lightgbm
kfold = StratifiedKFold(n_splits=10)
pred = np.zeros((len(test),9))
for index, (train_indices, val_indices) in enumerate(kfold.split(X_train, Y_train)):
    print("Training on fold " + str(index+1) + "/10...")

    xtrain, xval = X_train.iloc[train_indices], X_train.iloc[val_indices]
    ytrain, yval = Y_train[train_indices], Y_train[val_indices]
    
    train_data = lightgbm.Dataset(xtrain, label=ytrain)
    test_data = lightgbm.Dataset(xval, label=yval)
    
    model = lightgbm.train(para, train_data, valid_sets=test_data, num_boost_round=5000, early_stopping_rounds=50)
    pred += model.predict(test)
    print(pred.shape)
pred /= 10

Training on fold 1/10...
[1]	valid_0's multi_error: 0.6
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_error: 0.483117
[3]	valid_0's multi_error: 0.387013
[4]	valid_0's multi_error: 0.342857
[5]	valid_0's multi_error: 0.303896
[6]	valid_0's multi_error: 0.290909
[7]	valid_0's multi_error: 0.272727
[8]	valid_0's multi_error: 0.25974
[9]	valid_0's multi_error: 0.249351
[10]	valid_0's multi_error: 0.238961
[11]	valid_0's multi_error: 0.238961
[12]	valid_0's multi_error: 0.238961
[13]	valid_0's multi_error: 0.231169
[14]	valid_0's multi_error: 0.238961
[15]	valid_0's multi_error: 0.233766
[16]	valid_0's multi_error: 0.228571
[17]	valid_0's multi_error: 0.228571
[18]	valid_0's multi_error: 0.225974
[19]	valid_0's multi_error: 0.223377
[20]	valid_0's multi_error: 0.223377
[21]	valid_0's multi_error: 0.223377
[22]	valid_0's multi_error: 0.223377
[23]	valid_0's multi_error: 0.223377
[24]	valid_0's multi_error: 0.220779
[25]	valid_0's multi_error: 0.220779
[26

In [72]:
from sklearn.model_selection import GridSearchCV,StratifiedKFold, train_test_split
kfold = StratifiedKFold(n_splits=5)
from sklearn.metrics import roc_auc_score,roc_curve,auc
train_x,val_x,train_y,val_y = train_test_split(X_train, Y_train, test_size = 0.10, random_state=14)
train_x.shape,val_x.shape,train_y.shape,val_y.shape

((3429, 105), (381, 105), (3429,), (381,))

In [0]:
import lightgbm
train_data = lightgbm.Dataset(train_x, label=train_y)
test_data = lightgbm.Dataset(val_x, label=val_y)

In [74]:
para={'boosting_type': 'gbdt',
 'colsample_bytree': 0.85,
 'learning_rate': 0.1,
 'max_bin': 512,
 'max_depth': -1,
 'metric': 'multi_error',
 'min_child_samples': 8,
 'min_child_weight': 1,
 'min_split_gain': 0.5,
 'nthread': 3,
 'num_class': 9,
 'num_leaves': 31,
 'objective': 'multiclass',
 'reg_alpha': 0.8,
 'reg_lambda': 1.2,
 'scale_pos_weight': 1,
 'subsample': 0.7,
 'subsample_for_bin': 200,
 'subsample_freq': 1}

model = lightgbm.train(para,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=50,
                      )

[1]	valid_0's multi_error: 0.553806
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_error: 0.401575
[3]	valid_0's multi_error: 0.299213
[4]	valid_0's multi_error: 0.244094
[5]	valid_0's multi_error: 0.223097
[6]	valid_0's multi_error: 0.217848
[7]	valid_0's multi_error: 0.204724
[8]	valid_0's multi_error: 0.186352
[9]	valid_0's multi_error: 0.178478
[10]	valid_0's multi_error: 0.175853
[11]	valid_0's multi_error: 0.167979
[12]	valid_0's multi_error: 0.16273
[13]	valid_0's multi_error: 0.160105
[14]	valid_0's multi_error: 0.167979
[15]	valid_0's multi_error: 0.15748
[16]	valid_0's multi_error: 0.16273
[17]	valid_0's multi_error: 0.16273
[18]	valid_0's multi_error: 0.16273
[19]	valid_0's multi_error: 0.160105
[20]	valid_0's multi_error: 0.154856
[21]	valid_0's multi_error: 0.152231
[22]	valid_0's multi_error: 0.146982
[23]	valid_0's multi_error: 0.149606
[24]	valid_0's multi_error: 0.139108
[25]	valid_0's multi_error: 0.139108
[26]	valid_0's multi_error:

In [0]:
class_prediction=pd.DataFrame(pred).idxmax(axis=1) 

In [149]:
submission1 = pd.DataFrame({
        "series_id": test.index,
        "surface": class_prediction,
        
    })
submission1.surface.value_counts()

1    938
8    705
5    688
6    433
7    414
4    256
2    240
0    141
3      1
Name: surface, dtype: int64

In [150]:
submission1 = pd.DataFrame({
        "series_id": test.index,
        "surface": class_prediction,
        
    })
submission1.surface=label_encoder.inverse_transform(submission1.surface)
submission1

,series_id,surface
0,0,hard_tiles_large_space
1,1,carpet
2,2,tiled
3,3,soft_tiles
4,4,soft_tiles
5,5,concrete
6,6,concrete
7,7,concrete
8,8,wood
9,9,wood


In [0]:
submission1.to_csv('submission.csv', index=False)

In [154]:
!kaggle competitions submit -c career-con-2019 -f submission.csv -m "First submission"

100% 52.4k/52.4k [00:03<00:00, 15.9kB/s]
Successfully submitted to CareerCon 2019 - Help Navigate Robots 